Исаченко М.К. БПМ-19-2 Вариант 30

*   Длина слова с контрольными битами: 62
*   Алгоритм контрольной суммы: CRC-64

In [362]:
WORD_LEN = 56 # - количество значащих битов в слове

In [363]:
# %pip install crc64iso

In [364]:
from typing import List
from math import log2, ceil
from random import randrange
from crc64iso.crc64iso import crc64

In [365]:
def hamming_core(src: List[List[int]], s_num: int, encode=True) -> int:
    s_range = range(s_num)
    errors = 0

    for i in src:
        sindrome = 0
        for s in s_range:
            sind = 0
            for p in range(2 ** s, len(i) + 1, 2 ** (s + 1)):
                for j in range(2 ** s):
                    if (p + j) > len(i):
                        break
                    sind ^= i[p + j - 1]

            if encode:
                i[2 ** s - 1] = sind
            else:
                sindrome += (2 ** s * sind)

        if (not encode) and sindrome:
            try:
                i[sindrome - 1] = int(not i[sindrome - 1])
            except IndexError:
                errors += 1

    return errors

In [366]:
def hamming_encode(msg: str, mode: int = 8) -> str:
    result = ""
    msg_b = msg.encode("utf8")
    s_num = ceil(log2(log2(mode + 1) + mode + 1))
    bit_seq = []
    for byte in msg_b:
        bit_seq += list(map(int, f"{byte:08b}"))

    res_len = ceil((len(msg_b) * 8) / mode)
    bit_seq += [0] * (res_len * mode - len(bit_seq))

    to_hamming = []

    for i in range(res_len):
        code = bit_seq[i * mode:i * mode + mode]
        for j in range(s_num):
            code.insert(2 ** j - 1, 0)
        to_hamming.append(code)

    errors = hamming_core(to_hamming, s_num, True)

    for i in to_hamming:
        result += "".join(map(str, i))

    return result

In [367]:
def hamming_decode(msg: str, mode: int = 8):
    result = ""

    s_num = ceil(log2(log2(mode + 1) + mode + 1))
    res_len = len(msg) // (mode + s_num)
    code_len = mode + s_num

    to_hamming = []

    for i in range(res_len):
        code = list(map(int, msg[i * code_len:i * code_len + code_len]))
        to_hamming.append(code)

    errors = hamming_core(to_hamming, s_num, False)

    for i in to_hamming:
        for j in range(s_num):
            i.pop(2 ** j - 1 - j)
        result += "".join(map(str, i))

    msg_l = []

    for i in range(len(result) // 8):
        val = "".join(result[i * 8:i * 8 + 8])
        msg_l.append(int(val, 2))
    try:
        result = bytes(msg_l).decode("utf-8")
    except UnicodeDecodeError:
        pass

    return result, errors

In [368]:
def generate_simple_errors(msg: str, mode: int) -> str:
    seq = list(map(int, msg))
    s_num = ceil(log2(log2(mode + 1) + mode + 1))
    code_len = mode + s_num
    cnt = len(msg) // code_len
    result = ""

    for i in range(cnt):
        to_noize = seq[i * code_len:i * code_len + code_len]
        noize = randrange(code_len)
        to_noize[noize] = int(not to_noize[noize])
        result += "".join(map(str, to_noize))

    return result

In [369]:
def generate_complex_errors(msg: str, mode: int) -> str:
    seq = list(map(int, msg))
    s_num = ceil(log2(log2(mode + 1) + mode + 1))
    code_len = mode + s_num
    cnt = len(msg) // code_len
    result = ""

    for i in range(cnt):
        to_noize = seq[i * code_len:i * code_len + code_len]
        noize1 = randrange(code_len)
        noize2 = randrange(code_len)
        noize3 = randrange(code_len)
        to_noize[noize1] = int(not to_noize[noize1])
        to_noize[noize2] = int(not to_noize[noize2])
        to_noize[noize3] = int(not to_noize[noize3])
        result += "".join(map(str, to_noize))

    return result

In [370]:
message = '''В современном мире наряду со стремительным развитием интернет-технологий значительно увеличился уровень онлайн-мошенничества. В большинстве своем злоумышленники заинтересованы в получении конфиденциальных данных, например логина, пароля или номера банковской карты. Одним из способов достижения подобной цели является кибератака под названием «фишинг», в ходе которой жертва получает письмо или сообщение от имени популярной организации, например банка, содержащее ссылку на поддельный сайт внешне неотличимый от настоящего и побуждающий ввести логин и пароль. Введенная конфиденциальная информация перехватывается создателями фишингового веб-сайта и применяется для доступа к настоящему аккаунту пользователя.
Фишинг является простой и одной из самых опасных атак, для осуществления которой не требуются глубокие технические знания, ведь объектом атаки выступает человеческий разум, на который оказывается воздействие с помощью техник социальной инженерии. В силу своей эффективности, число фишинговых ресурсов растет с каждым годом. Так, по данным компании «SlashNext» [1], в 2022 году было обнаружено более 255 миллионов фишинговых атак, что на 61% больше чем в 2021. Поскольку количество фишинговых атак постоянно растет, существует острая необходимость в эффективном способе обнаружения фишинга, благодаря которому жертвы могут быть защищены от потенциальной потери конфиденциальных данных.
Обнаружить фишинговую атаку можно как на этапе получения письма или сообщения, проанализировав приложенный текст, так и за счет работы с URL-адресом и содержимым веб-страницы фишингового сайта. Данная КНИР будет сфокусирована на втором варианте. Для распознавания фишинговых веб-сайтов существуют разные методы, основанные на черных списках, наборах правил, визуальном сходстве и машинном обучении. В данной работе будет сделан упор на именно на методах машинного обучения в силу их большей устойчивости к возможным возникновениям новых вариаций данной кибератаки.
Глобально, детекция фишингового веб-сайта на основе его URL-адреса и контента с помощью машинного обучения сводится к решению задачи бинарной классификаци. В ходе данной научно-исследовательской работы был проведен аналитический обзор литературы, наиболее популярные алгоритмы, применяющиеся для задачи распознавания фишинговых веб-сайтов. Выявлена актуальность рассматриваемой проблемы. При изучении методов внимание уделялось не только их идейному содержанию, но и математическому обоснованию их работы. Также были изучены основные функции оценки моделей бинарной классификации и выбраны те их них, по которым будет проводиться сравнение. Помимо обзора литературы, была составлена блок-схема будущего алгоритма сравнения методов. Также представлен концепт программной системы для защиты от фишинговых атак в которую будет интегрирована отобранная модель. Для разработкивперечисленного определены программные средства. 
При   проведении   данной   научно-исследовательской   работы   основную   часть математического аппарата составили элементы теории вероятности для обоснования работы самих методов. Основным направлением для продолжения работы станет непосредственно создание программного обеспечения для обучения и тестирования моделей и подсчета соответствующих метрик, а также разработка предложенной системы защиты от фишинговых атак'''
print('Исходное сообщение:')
message

Исходное сообщение:


'В современном мире наряду со стремительным развитием интернет-технологий значительно увеличился уровень онлайн-мошенничества. В большинстве своем злоумышленники заинтересованы в получении конфиденциальных данных, например логина, пароля или номера банковской карты. Одним из способов достижения подобной цели является кибератака под названием «фишинг», в ходе которой жертва получает письмо или сообщение от имени популярной организации, например банка, содержащее ссылку на поддельный сайт внешне неотличимый от настоящего и побуждающий ввести логин и пароль. Введенная конфиденциальная информация перехватывается создателями фишингового веб-сайта и применяется для доступа к настоящему аккаунту пользователя.\nФишинг является простой и одной из самых опасных атак, для осуществления которой не требуются глубокие технические знания, ведь объектом атаки выступает человеческий разум, на который оказывается воздействие с помощью техник социальной инженерии. В силу своей эффективности, число фишинг

In [371]:
checksum = crc64(message)
print('Контрольная сумма:')
checksum

Контрольная сумма:


'5E7CDED5F6C66CE8'

In [372]:
encoded_message = hamming_encode(message, WORD_LEN)
print("Закодированное сообщение:")
encoded_message

Закодированное сообщение:


'...'

#### Отправка без ошибок

In [373]:
decoded_message, err = hamming_decode(encoded_message, WORD_LEN)

print('Раскодированное сообщение:')
decoded_message

Раскодированное сообщение:


'В современном мире наряду со стремительным развитием интернет-технологий значительно увеличился уровень онлайн-мошенничества. В большинстве своем злоумышленники заинтересованы в получении конфиденциальных данных, например логина, пароля или номера банковской карты. Одним из способов достижения подобной цели является кибератака под названием «фишинг», в ходе которой жертва получает письмо или сообщение от имени популярной организации, например банка, содержащее ссылку на поддельный сайт внешне неотличимый от настоящего и побуждающий ввести логин и пароль. Введенная конфиденциальная информация перехватывается создателями фишингового веб-сайта и применяется для доступа к настоящему аккаунту пользователя.\nФишинг является простой и одной из самых опасных атак, для осуществления которой не требуются глубокие технические знания, ведь объектом атаки выступает человеческий разум, на который оказывается воздействие с помощью техник социальной инженерии. В силу своей эффективности, число фишинг

In [374]:
print('Контрольная сумма:')
crc64(decoded_message)

Контрольная сумма:


'5E7CDED5F6C66CE8'

In [375]:
print('Контрольная сумма раскодированного сообщения равна контрольной сумме исходного:')
crc64(decoded_message) == checksum

Контрольная сумма раскодированного сообщения равна контрольной сумме исходного:


True

#### Отправка с неболее одной ошибкой на слово

In [376]:
message_with_errors = generate_simple_errors(encoded_message, WORD_LEN)
print('Кодированное сообщение с ошибками:')
message_with_errors

Кодированное сообщение с ошибками:


'...'

In [377]:
decoded_message, err = hamming_decode(message_with_errors, WORD_LEN)

print('Раскодированное сообщение:')
decoded_message

Раскодированное сообщение:


'В современном мире наряду со стремительным развитием интернет-технологий значительно увеличился уровень онлайн-мошенничества. В большинстве своем злоумышленники заинтересованы в получении конфиденциальных данных, например логина, пароля или номера банковской карты. Одним из способов достижения подобной цели является кибератака под названием «фишинг», в ходе которой жертва получает письмо или сообщение от имени популярной организации, например банка, содержащее ссылку на поддельный сайт внешне неотличимый от настоящего и побуждающий ввести логин и пароль. Введенная конфиденциальная информация перехватывается создателями фишингового веб-сайта и применяется для доступа к настоящему аккаунту пользователя.\nФишинг является простой и одной из самых опасных атак, для осуществления которой не требуются глубокие технические знания, ведь объектом атаки выступает человеческий разум, на который оказывается воздействие с помощью техник социальной инженерии. В силу своей эффективности, число фишинг

In [378]:
print('Контрольная сумма:')
crc64(decoded_message)

Контрольная сумма:


'5E7CDED5F6C66CE8'

In [379]:
print('Контрольная сумма раскодированного сообщения равна контрольной сумме исходного:')
crc64(decoded_message) == checksum

Контрольная сумма раскодированного сообщения равна контрольной сумме исходного:


True

In [380]:
print('Количество обнаруженных ошибок:')
err

Количество обнаруженных ошибок:


0

#### Отправка с множественными ошибками

In [381]:
message_with_errors = generate_complex_errors(encoded_message, WORD_LEN)
print('Кодированное сообщение с ошибками:')
message_with_errors

Кодированное сообщение с ошибками:


'...'

In [382]:
decoded_message, err = hamming_decode(message_with_errors, WORD_LEN)

print(f'Раскодированное сообщение:')
decoded_message

Раскодированное сообщение:


'...'

In [383]:
print('Контрольная сумма:')
crc64(decoded_message)

Контрольная сумма:


'78F8A2E4D2DEFE7C'

In [384]:
print('Контрольная сумма раскодированного сообщения равна контрольной сумме исходного:')
crc64(decoded_message) == checksum

Контрольная сумма раскодированного сообщения равна контрольной сумме исходного:


False

In [385]:
print('Количество обнаруженных ошибок:')
err

Количество обнаруженных ошибок:


10